In [1]:
from owlready2 import *
import rdflib
import os

import numpy as np
import random
from copy import copy
import pickle
import pandas as pd

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# PATH = "../../../../../../Documents/School/Graduate/Year 2/Fall/CS 692A/Project/datasets/anatomy-dataset/anatomy-dataset/"
# PATH = "C:/Users/evan_/OneDrive/Documents/School/Graduate/Year 2/Fall/CS 692A/Project/datasets/anatomy-dataset/anatomy-dataset/"
PATH = "../data/"

MOUSEFILE = 'mouse.owl'
HUMANFILE = 'human.owl'
REFFILE   = 'reference.rdf'
#HM_ALIGN_SCORE = 'human_mouse_alignment_score.csv'
HM_ALIGN_SCORE = 'am_alignment_score.txt'

In [3]:
HM_ALIGN_SCORE_list = []
for line in open(PATH+HM_ALIGN_SCORE):  
    HM_ALIGN_SCORE_list.append(line.strip('\n'))
    
am_mouse_entity = HM_ALIGN_SCORE_list[1:2745]
am_human_entity = HM_ALIGN_SCORE_list[2746:6050]
align_score = HM_ALIGN_SCORE_list[6051:]

## Loading both ontologies

In [4]:
mouse_onto = get_ontology(PATH+MOUSEFILE).load()
human_onto = get_ontology(PATH+HUMANFILE).load()

mc = list(mouse_onto.classes())
hc = list(human_onto.classes())

In [5]:
mouse_entities = {}
human_entities = {}
entities       = {}
entities_names = {}
labels  = {}

# We need to save the object 'owl.Thing' in the entity dictionaries, 
# Find the owl.Thing object (typically in position 0) and save its position within the mouse classes
for i, m in enumerate(mc):
    if m is owl.Thing:
        ot = i
        print("owl.Thing is located at position:", ot)
        break

# insert the owl.Thing object as the first object within the dictionary
mouse_entities[mc[ot]]      = 0
entities[mc[ot]]            = 0
human_entities[mc[ot]]      = 0
entities_names[mc[ot].name] = 0
count_mc = 0
count_hc = 0

for c in mc:
    if "MA" in c.name: # filter only the classes, prevent "OboInOwl" types being included in the dictionary
        if c not in entities:
            labels[c.label[0]] = len(entities)
            mouse_entities[c] = len(entities)
            entities[c] = len(entities)
            entities_names[c.name] = len(entities_names)
        
    count_mc+=1
            

for c in hc:
    if "NCI" in c.name: # filter only the classes, prevent "OboInOwl" types being included in the dictionary
        if c not in entities:
            labels[c.label[0]] = len(entities)
            human_entities[c] = len(entities)
            entities[c] = len(entities)
            entities_names[c.name] = len(entities_names)
        
    count_hc+=1
            
idx_to_labels = dict((v,k) for k,v in labels.items())
idx_to_entity = dict((v,k) for k,v in entities.items())
idx_to_entity_names = dict((v,k) for k,v in entities_names.items())
idx_to_mouse_entity = dict((v,k) for k,v in mouse_entities.items())
idx_to_human_entity = dict((v,k) for k,v in human_entities.items())

owl.Thing is located at position: 0


In [6]:
entities_2 = {}
for item in entities:
    #print(item, str(item).split('.')[1], entities[item])
    entities_2[str(item).split('.')[1]] = entities[item]

In [7]:
am_mouse_entity_id = []
for item in am_mouse_entity:
    try:
        am_mouse_entity_id.append(entities_2[item.split('#')[1]])
    except:
        am_mouse_entity_id.append(item.split('#')[1])
        
am_human_entity_id = []
for item in am_human_entity:
    try:
        am_human_entity_id.append(entities_2[item.split('#')[1]])
    except:
        am_human_entity_id.append(item.split('#')[1])

In [47]:
# am_mouse_entity
# am_human_entity
# am_mouse_entity_id
# am_human_entity_id
# align_score

In [13]:
def partition(ls, size):
    """
    Returns a new list with elements
    of which is a list of certain size.

        >>> partition([1, 2, 3, 4], 3)
        [[1, 2, 3], [4]]
    """
    return [ls[i:i+size] for i in range(0, len(ls), size)]

align_score_partition = partition(align_score, 3304)

In [24]:
df = pd.DataFrame(data=align_score_partition, index=am_mouse_entity_id, columns=am_human_entity_id)

In [34]:
df = df.drop(['DbXref', 'Definition', 'ObsoleteClass', 'Subset', 'Synonym', 'SynonymType', 0])
df = df.drop(['DbXref', 'Definition', 'ObsoleteClass', 'Subset', 'Synonym', 'SynonymType'], axis=1)
df

,2738,2739,2742,2744,2743,2745,2747,2746,2749,2750,...,6027,6028,6029,6030,6031,6032,6033,6034,5809,6035
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25866786329274394
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.11625552234800896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df.to_csv('../data/ontologies/anatomy/human_mouse_alignment_score.tsv')

In [46]:
#pd.read_csv('../data/ontologies/anatomy/human_mouse_alignment_score.tsv', index_col=0)

,2738,2739,2742,2744,2743,2745,2747,2746,2749,2750,...,6027,6028,6029,6030,6031,6032,6033,6034,5809,6035
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.258668
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.116256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2734,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2735,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2736,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


## Saving edges within the ontologies

In [48]:
def set_dict(_dict:dict, key:int, value:int):
    if key in _dict:
        _dict[key].append(value) 
    else:
        _dict[key] = [value]


def get_edges(entities:dict, USE_UNDEFINED_EDGES:bool=True, USE_OWL_THING:bool=False):
    # _edges will contain a list of tuples of all the edges in an ontology
    # _parents is a dictionary of all the parents of a given key
    # _children is a dictionary of all the children of a given key
    _edges    = []
    _parents  = {}
    _children = {}
    
    # loop over all of the entities in the dictionary of human entities
    for e in entities: 
        _node1 = entities[e]
        
        # loop over the entity's 'is_a' relations
        for i in e.is_a:
            
            # this condition is for UNDEFINED edges
            if hasattr(i, 'value') and USE_UNDEFINED_EDGES: 
                _node2 = entities[i.value]
            
            # if we are not using "UNDEFINED_part_of" relations -- skip this relation
            elif hasattr(i, 'value') and not USE_UNDEFINED_EDGES:
                continue
                
            # this condition is for subClassOf relations
            else: 
                _node2 = entities[i]
                
            # "owl.Thing" is set to index 0
            # if _node2 is 0, and we are not using the "owl.Thing" object -- skip this relation 
            if _node2 == 0 and not USE_OWL_THING:
                continue
                
            # add the edge between two nodes to a list of edges                
            _edges.append((_node2, _node1))

            # add the edge to a dictionary that contains all the parents and children
            set_dict(_parents, _node1, _node2)
            set_dict(_children, _node2, _node1)
                
    return _edges, _parents, _children
                
h_edges, h_parents, h_children = get_edges(human_entities)
m_edges, m_parents, m_children = get_edges(mouse_entities)

# _parents = {**m_parents, **h_parents}
# _children = {**m_children, **h_children}

## Load the reference (.rdf)

## The following two code blocks may not be the most efficient method of extracting the data... but it works

In [49]:
g = rdflib.Graph()
g.load(PATH+REFFILE)

# Create a list of triples that contain:
# (Subject, Predicate, Object)
SPOtriple = []

for sub, pred, obj in g:
    SPOtriple.append((sub, pred, obj))

Each subject is contained in a number of different triples.  

Each "alignment subject" part of the triple seems to be some sort of identifier for each cell:

```html
<Cell>
    <entity1 ... />
    <entity2 ... />
    <measure ... />
    <relation ... />
</Cell>
```

Each alignment subject is essentially just the definition of an edge between two nodes. So each identifier can be used to link two nodes. Meaning you can use this subject identifier to find the nodes that belong to this edge.

In [50]:
alignment_subjects = {}
alignments  = []

# Make a count of how often subjects occur in this list of triples
for t in SPOtriple:
    if t[0] in alignment_subjects:
        alignment_subjects[t[0]] += 1
    else:
        alignment_subjects[t[0]] = 1

# Loop through every alignment subject (one alignment between two nodes)
for i, key in enumerate(alignment_subjects):
    # Make a list to get the two aligned nodes
    alignment_pair = [] 
    
    # Search for the two objects that contain the same subject alignment number
    for t in SPOtriple:
        if key in t[0]:
            if "alignmententity" in t[1]:
                # Save the index of the entity to the alignment pair 
                get_iri = human_onto.search(iri=t[2])[0]
                alignment_pair.append(entities[get_iri])
    
    # There are some SPOtriples that do not actually contain an alignment, so we ignore those.
    if alignment_pair:
        # Save 'is_a' relations in both directions, as alignments are bi-directional
        alignments.append(tuple(alignment_pair))
        alignments.append(tuple(alignment_pair[::-1])) # saves the reverse of the tuple
                

In [51]:
all_edges = m_edges + h_edges + alignments

print("edges:", len(all_edges))
print("alignment edges:", len(alignments))
print("mouse edges:", len(m_edges))
print("human edges:", len(h_edges))
print("entities:", len(entities))

edges: 11899
alignment edges: 3032
mouse edges: 3444
human edges: 5423
entities: 6036


In [53]:
alignments

[(400, 2972),
 (2972, 400),
 (512, 4786),
 (4786, 512),
 (3508, 2581),
 (2581, 3508),
 (773, 5588),
 (5588, 773),
 (1515, 5307),
 (5307, 1515),
 (4294, 2),
 (2, 4294),
 (2990, 432),
 (432, 2990),
 (4082, 1493),
 (1493, 4082),
 (146, 2816),
 (2816, 146),
 (157, 4745),
 (4745, 157),
 (6000, 2293),
 (2293, 6000),
 (3053, 227),
 (227, 3053),
 (1251, 4600),
 (4600, 1251),
 (2376, 4469),
 (4469, 2376),
 (1917, 5594),
 (5594, 1917),
 (5770, 917),
 (917, 5770),
 (821, 5207),
 (5207, 821),
 (128, 4409),
 (4409, 128),
 (112, 3435),
 (3435, 112),
 (3006, 214),
 (214, 3006),
 (4155, 1670),
 (1670, 4155),
 (2506, 4534),
 (4534, 2506),
 (1432, 5785),
 (5785, 1432),
 (5411, 243),
 (243, 5411),
 (249, 3990),
 (3990, 249),
 (1847, 4008),
 (4008, 1847),
 (5837, 1451),
 (1451, 5837),
 (5235, 1030),
 (1030, 5235),
 (3582, 856),
 (856, 3582),
 (5768, 1210),
 (1210, 5768),
 (1486, 5790),
 (5790, 1486),
 (2379, 5733),
 (5733, 2379),
 (3280, 295),
 (295, 3280),
 (207, 3068),
 (3068, 207),
 (71, 2968),
 (2968,

In [57]:
df[5588][773]

'0.7790807252211779'

In [58]:
df

,2738,2739,2742,2744,2743,2745,2747,2746,2749,2750,...,6027,6028,6029,6030,6031,6032,6033,6034,5809,6035
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25866786329274394
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.11625552234800896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Dictionary for alignments. Easy look-up

In [70]:
alignment_dict = {}

for a in alignments:
    if a[0] not in alignment_dict:
        alignment_dict[a[0]] = [a[1]]
    else:
        alignment_dict[a[0]].append(a[1])

In [71]:
human_set = set(idx_to_human_entity)
mouse_set = set(idx_to_mouse_entity)
entity_set = set(idx_to_entity)

In [72]:
human_pickle = {}
mouse_pickle = {}
entity_pickle = {}

human_pickle['edges'] = h_edges 
human_pickle['parents_of'] = h_parents
human_pickle['children_of'] = h_children
human_pickle['human_entities'] = human_set

mouse_pickle['edges'] = m_edges
mouse_pickle['parents_of'] = m_parents
mouse_pickle['children_of'] = m_children
mouse_pickle['mouse_entities'] = mouse_set

entity_pickle['all_edges'] = all_edges
entity_pickle['alignments'] = alignments
entity_pickle['name2idx'] = entities_names
entity_pickle['idx2name'] = idx_to_entity_names
entity_pickle['label2idx'] = labels
entity_pickle['idx2label'] = idx_to_labels
entity_pickle['set'] = entity_set
entity_pickle['align_dict'] = alignment_dict

with open('../data/ontologies/anatomy/human.pickle', 'wb') as handle:
    pickle.dump(human_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../data/ontologies/anatomy/mouse.pickle', 'wb') as handle:
    pickle.dump(mouse_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../data/ontologies/anatomy/entities.pickle', 'wb') as handle:
    pickle.dump(entity_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)    

## Generate all positive examples for the training dataset

### We get all of the edges within the respective trees, and use those as positive edges for training.
### We also take a ratio of the alignments between trees to include in the training dataset as positive edges

In [73]:
tr_m_pos_edges = []
tr_h_pos_edges = []

# Trainsplit: used to determine how many edges within a tree are going to be in the training set.
# 1.0 -> full set of edges in the ontology will be in the training dataset
train_split = 1.0

# alignmentsplit: choose how many of the alignment edges to include in the training dataset
alignment_split = 0.8

for edge in m_edges:
    if np.random.uniform() > (1-train_split):
        tr_m_pos_edges.append(edge)
    
for edge in h_edges:
    if np.random.uniform() > (1-train_split):
        tr_h_pos_edges.append(edge)

if np.floor(len(alignments)*alignment_split)%2 == 1.0:
    tr_pos_alignments  = alignments[:int(np.floor(len(alignments)*alignment_split))+1]
    dev_pos_alignments = alignments[int(np.floor(len(alignments)*alignment_split))+1:]
else:
    tr_pos_alignments  = alignments[:int(np.floor(len(alignments)*alignment_split))]
    dev_pos_alignments = alignments[int(np.floor(len(alignments)*alignment_split)):]

print("Number of training edges in the mouse ontology:", len(tr_m_pos_edges))
print("Number of training edges in the human ontology:", len(tr_h_pos_edges))

train_positives = tr_m_pos_edges + tr_h_pos_edges + tr_pos_alignments

Number of training edges in the mouse ontology: 3444
Number of training edges in the human ontology: 5423


In [76]:
with open('../data/ontologies/anatomy/alignment_data.pkl', 'wb') as f:
    pickle.dump(alignments, f)

## Generate negative examples for the dev set

In [65]:
def get_siblings(parents:dict, children:dict, node:int):
    siblings = []
    
    # There should only be only one node that doesn't have any parents, the root node
    if node in parents:
        parents_of_node = parents[node]
        
        # Cycle through all possible parents of the given node
        for p in parents_of_node:
            
            # if the parent node has any children, add them to the siblings list
            if p in children:
                siblings = siblings + children[p]
                
                # remove the node from the siblings list
                siblings.remove(node)
        
        # if there are any siblings, return the list of them
        if siblings:
            return siblings
        
        # if there are no siblings, return -1
        else:
            print("Given node does not have any siblings:", node)
            return -1
            
    # if the node does not have any parents, return -1
    else:
        print("Given node does not have any parents:", node)
        return -1
    # ---- 

def generate_true_neg_alignments(alignments:list, alignment_split:float=0.5, ratio:float=1.0):
    
    true_negatives = []
    numFailures = 0
    num_samples = int(len(alignments) * alignment_split * ratio)
    
    while (len(true_negatives) < num_samples) and (numFailures < 100):
        # Select a random alignment within the list of all alignments
        rdm_align = random.choice(alignments)

        # Pick a node to alter within the randomly chosen alignment 
        const_node = rdm_align[0]
        change_node = rdm_align[1]
        
        # generate all siblings within the human ontology of the chosen node
        if change_node in h_parents:
            siblings = get_siblings(h_parents, h_children, change_node)
            
        # generate all siblings within the mouse ontology of the chosen node
        elif change_node in m_parents:
            siblings = get_siblings(m_parents, m_children, change_node)
            
        # This shouldn't be triggered -- every node should have a parent node
        # The only possible node that could trigger the below statement is the root node
        else:
            print("Node not found in either Ontology or does not have any parents")
            
            
        # This error will typically be thrown if the chosen node does not have any siblings
        if siblings == -1:
            print("Error thrown when retrieving siblings")
            
        else:
            # Choose some random siblings to be make the true negative
            negative_alignment = (const_node, random.choice(siblings))
            
            if negative_alignment in alignments:
                numFailures += 1
                print("Generated negative is an existing alignment:", negative_alignment, "OG random:", rdm_align, siblings)
                pass
            
            elif negative_alignment in true_negatives:
                numFailures += 1
                print("Generated negative already in true_negatives:", negative_alignment)
                pass
            
            # include this negative alignment in the true_negatives list
            else:
                true_negatives.append(negative_alignment)
                true_negatives.append((negative_alignment[1], const_node))
                numFailures = 0
                
            
    return true_negatives

In [14]:
tr_neg_alignments = generate_true_neg_alignments(alignments, alignment_split)

Given node does not have any siblings: 6010
Error thrown when retrieving siblings
Given node does not have any siblings: 1818
Error thrown when retrieving siblings
Given node does not have any siblings: 2529
Error thrown when retrieving siblings
Given node does not have any siblings: 2529
Error thrown when retrieving siblings
Given node does not have any siblings: 1590
Error thrown when retrieving siblings
Given node does not have any siblings: 5778
Error thrown when retrieving siblings
Given node does not have any siblings: 1138
Error thrown when retrieving siblings
Given node does not have any siblings: 5584
Error thrown when retrieving siblings
Generated negative already in true_negatives: (4764, 1294)
Generated negative already in true_negatives: (4308, 1659)
Generated negative already in true_negatives: (2556, 4323)
Given node does not have any siblings: 5762
Error thrown when retrieving siblings
Generated negative already in true_negatives: (3665, 311)
Given node does not have an

In [15]:
np.savetxt(f'../data/ontologies/anatomy/tr_align_pos_{alignment_split}.tsv', tr_pos_alignments, delimiter='\t', fmt='%1.1d')
np.savetxt(f'../data/ontologies/anatomy/tr_align_neg_{alignment_split}.tsv', tr_neg_alignments, delimiter='\t', fmt='%1.1d')

In [16]:
dev_true_negatives = generate_true_neg_alignments(alignments, (1-alignment_split))

Given node does not have any siblings: 2528
Error thrown when retrieving siblings
Given node does not have any siblings: 1651
Error thrown when retrieving siblings
Given node does not have any siblings: 1924
Error thrown when retrieving siblings
Given node does not have any siblings: 1839
Error thrown when retrieving siblings
Generated negative already in true_negatives: (4246, 1712)
Given node does not have any siblings: 5778
Error thrown when retrieving siblings
Generated negative already in true_negatives: (817, 5459)
Given node does not have any siblings: 5745
Error thrown when retrieving siblings
Given node does not have any siblings: 1606
Error thrown when retrieving siblings
Generated negative already in true_negatives: (2546, 3779)
Given node does not have any siblings: 2010
Error thrown when retrieving siblings
Generated negative already in true_negatives: (877, 4310)
Generated negative already in true_negatives: (5649, 1877)


## Generate negative samples for the alignment training dataset (not used - we will be generating negatives at training time)

In [17]:
negatives_ratio = 10

tr_m_neg_edges = np.random.choice(list(idx_to_mouse_entity.keys()), size=(int(negatives_ratio*len(tr_m_pos_edges)), 2))
tr_h_neg_edges = np.random.choice(list(idx_to_human_entity.keys()), size=(int(negatives_ratio*len(tr_h_pos_edges)), 2))

train_negatives = list(tr_m_neg_edges) + list(tr_h_neg_edges)

In [18]:
np.savetxt(f'../data/ontologies/anatomy/tr_pos_{alignment_split}.tsv', train_positives, delimiter='\t', fmt='%1.1d')
np.savetxt(f'../data/ontologies/anatomy/tr_neg_{alignment_split}.tsv', train_negatives, delimiter='\t', fmt='%1.1d')
np.savetxt(f'../data/ontologies/anatomy/dev_align_pos_{alignment_split}.tsv', dev_pos_alignments, delimiter='\t', fmt='%1.1d')
np.savetxt(f'../data/ontologies/anatomy/dev_align_neg_{alignment_split}.tsv', dev_true_negatives, delimiter='\t', fmt='%1.1d')

In [19]:
def get_all_parents(parents_of:dict, node:int):
    ancestors = []
    
    if node in parents_of:
        ancestors = ancestors + parents_of[node]
        
        for a in ancestors:
            ancestors = ancestors + get_all_parents(parents_of, a)
            
    else:
        pass
    
    return list(set(ancestors))

def get_all_children(children_of:dict, node:int):
    descendants = []
    
    if node in children_of:
        # print(node, children_of[node])
        descendants = descendants + children_of[node]
        
        for d in descendants:
            descendants = descendants + get_all_children(children_of, d)
        # print("returning from", node, descendants)
            
    else:
        # print(node, "has no children")
        pass
    
    return list(set(descendants))

## Transitive Closure

In [20]:
def nodes_in_tr_align(tr_alignments):
    tr_align_nodes = []
    
    for edge in tr_alignments:
        for node in edge:
            tr_align_nodes.append(node)
    
    return set(tr_align_nodes)    

tr_align_set = nodes_in_tr_align(tr_pos_alignments)

In [21]:
def transitive_closure(entity_set:set, h_parents, m_parents, tr_alignments, USE_OWL_THING:bool=False):
    transitive_edges = []
    mouse_tc = []
    human_tc = []

    for _node1 in entity_set:
        
        if _node1 in mouse_set:
            parents = get_all_parents(m_parents, _node1)       
            other_parents = h_parents
        
        elif _node1 in human_set:
            parents = get_all_parents(h_parents, _node1)
            other_parents = m_parents 
        
        if not parents:
            continue
        
        align_parents = []
        # go through all ancestors of the current node (_node1)
        for p in parents:
            
            # if a parent of the node has an alignment, get the parents of that alignment
            if p in tr_alignments:
                # since some nodes can have multiple alignments, 
                # go through every alignment and add all parents to the list
                for aligned_node in alignment_dict[p]:
                    align_parents.append(aligned_node)
                    align_parents = align_parents + get_all_parents(other_parents, aligned_node)
                    
        parents = parents + align_parents
        
        for _node2 in parents:
            transitive_edges.append((_node2, _node1))
            
            if _node1 in mouse_set:
                mouse_tc.append((_node2, _node1))
                
            elif _node1 in human_set:
                human_tc.append((_node2, _node1))
        
    return transitive_edges, mouse_tc, human_tc

tc_pos_edges, mouse_tc, human_tc = transitive_closure(entity_set, h_parents, m_parents, tr_align_set)

In [22]:
tc_negatives_ratio = 2

tc_m_neg_edges = np.random.choice(list(idx_to_mouse_entity.keys()), size=(int(tc_negatives_ratio*len(mouse_tc)), 2))
tc_h_neg_edges = np.random.choice(list(idx_to_human_entity.keys()), size=(int(tc_negatives_ratio*len(human_tc)), 2))

tc_neg_edges = list(tc_m_neg_edges) + list(tc_h_neg_edges)

np.savetxt(f'../data/ontologies/anatomy/tr_pos_tc_{alignment_split}.tsv', tc_pos_edges, delimiter='\t', fmt='%1.1d')
np.savetxt(f'../data/ontologies/anatomy/tr_neg_tc_{alignment_split}.tsv', tc_neg_edges, delimiter='\t', fmt='%1.1d')

# Everything below this is just looking into the ontologies separately

## The below three blocks are for generating data for the alignment demonstration. This data will be used to see if the model is learning relations within each individual tree.

In [23]:
def generate_dev_negatives(idx_of_entities:dict, parents_of:dict, children_of:dict, ratio:float, num_positives:int):
    """Generate dev negatives for single ontologies"""
    
    _entities = list(idx_of_entities)
    _dev_negatives = []
    
    # We don't want the index 0 (owl.Thing) to be a negative example
    _entities.remove(0)
    
    _num_negatives = ratio * num_positives
    
    while len(_dev_negatives) < _num_negatives:
        _node1 = random.choice(_entities)

        _descendants = get_all_children(children_of, _node1)
        _ancestors   = get_all_parents(parents_of, _node1)

        _remove_nodes = [_node1] + _ancestors + _descendants

        # remove any entity that is in remove_nodes
        _possible_negatives = [x for x in _entities if x not in _remove_nodes]
        
        if len(_possible_negatives) == 0:
            continue
        
        _node2 = random.choice(_possible_negatives)

        _dev_negatives.append((_node2, _node1))
        
    return _dev_negatives

In [24]:
human_align_dev_neg = generate_dev_negatives(idx_to_human_entity, h_parents, h_children, 1.0, len(tr_h_pos_edges))
mouse_align_dev_neg = generate_dev_negatives(idx_to_mouse_entity, m_parents, m_children, 1.0, len(tr_m_pos_edges))

In [25]:
np.savetxt('../data/ontologies/anatomy/human_dev_pos.tsv', tr_h_pos_edges, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/human_dev_neg.tsv', human_align_dev_neg, delimiter='\t', fmt='%1.1d')

np.savetxt('../data/ontologies/anatomy/mouse_dev_pos.tsv', tr_m_pos_edges, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/mouse_dev_neg.tsv', mouse_align_dev_neg, delimiter='\t', fmt='%1.1d')

## Create individual datasets for human and mouse (these are to not be used for the alignment task, just to ensure that boxes can learn these ontologies)

In [26]:
mouse_only_tr_pos = []
human_only_tr_pos = []
mouse_only_dev_pos = []
human_only_dev_pos = []

single_onto_split = 0.8

for edge in m_edges:
    if np.random.uniform() > (1-single_onto_split):
        mouse_only_tr_pos.append(edge)
    else:
        mouse_only_dev_pos.append(edge)
    
for edge in h_edges:
    if np.random.uniform() > (1-single_onto_split ):
        human_only_tr_pos.append(edge)
    else:
        human_only_dev_pos.append(edge)

In [27]:
def generate_tr_negatives(idx_of_entities:dict, ratio:float, num_positives:int):
    _entities = list(idx_of_entities)
    
    # We don't want the index 0 (owl.Thing) to be a negative example
    _entities.remove(0)
    
    _num_negatives = ratio * num_positives
    
    _tr_negatives = np.random.choice(_entities, size=(int(ratio*num_positives), 2))
    
    return _tr_negatives

In [28]:
negatives_ratio = 1

human_only_dev_neg = generate_dev_negatives(idx_to_human_entity, h_parents, h_children, negatives_ratio, len(human_only_dev_pos))    
mouse_only_dev_neg = generate_dev_negatives(idx_to_mouse_entity, m_parents, m_children, negatives_ratio, len(mouse_only_dev_pos))    

human_only_tr_neg = generate_tr_negatives(idx_to_human_entity, negatives_ratio, len(human_only_tr_pos))    
mouse_only_tr_neg = generate_tr_negatives(idx_to_mouse_entity, negatives_ratio, len(mouse_only_tr_pos))    

## Adjust the data so that is ready for the model (put it in the range (0, size of ontology)

In [29]:
mouse_only_adj_tr_pos  = np.asarray(mouse_only_tr_pos) - 1
mouse_only_adj_tr_neg  = mouse_only_tr_neg - 1 
mouse_only_adj_dev_pos = np.asarray(mouse_only_dev_pos) - 1
mouse_only_adj_dev_neg = np.asarray(mouse_only_dev_neg) - 1

human_only_adj_tr_pos  = np.asarray(human_only_tr_pos) - len(mouse_entities)  
human_only_adj_tr_neg  = human_only_tr_neg - len(mouse_entities)
human_only_adj_dev_pos = np.asarray(human_only_dev_pos) - len(mouse_entities) 
human_only_adj_dev_neg = np.asarray(human_only_dev_neg) - len(mouse_entities)

In [30]:
np.savetxt('../data/ontologies/anatomy/individual_analysis/human_adj_tr_pos.tsv', human_only_adj_tr_pos, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/individual_analysis/human_adj_tr_neg.tsv', human_only_adj_tr_neg, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/individual_analysis/human_adj_dev_pos.tsv', human_only_adj_dev_pos, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/individual_analysis/human_adj_dev_neg.tsv', human_only_adj_dev_neg, delimiter='\t', fmt='%1.1d')

np.savetxt('../data/ontologies/anatomy/individual_analysis/mouse_adj_tr_pos.tsv', mouse_only_adj_tr_pos, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/individual_analysis/mouse_adj_tr_neg.tsv', mouse_only_adj_tr_neg, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/individual_analysis/mouse_adj_dev_pos.tsv', mouse_only_adj_dev_pos, delimiter='\t', fmt='%1.1d')
np.savetxt('../data/ontologies/anatomy/individual_analysis/mouse_adj_dev_neg.tsv', mouse_only_adj_dev_neg, delimiter='\t', fmt='%1.1d')

In [31]:
mouse_probs = [1/(len(idx_to_mouse_entity)-1) for i in range(len(idx_to_mouse_entity)) ]
human_probs = [1/(len(idx_to_human_entity)-1) for i in range(len(idx_to_human_entity)) ]

unary_probs = mouse_probs + human_probs - 1
unary_probs = mouse_probs + human_probs #???should - 1 for two #Thing:0?

In [32]:
np.savetxt('../data/ontologies/anatomy/unary/human_unary.tsv', human_probs, delimiter='\t')
np.savetxt('../data/ontologies/anatomy/unary/mouse_unary.tsv', mouse_probs, delimiter='\t')
np.savetxt('../data/ontologies/anatomy/unary/unary.tsv', unary_probs, delimiter='\t')

## naive method of getting negatives

In [ ]:
# mouse_only_tr_neg = np.random.choice(list(idx_to_mouse_entity), size=(int(negatives_ratio*len(mouse_only_tr_pos)), 2))
# human_only_tr_neg = np.random.choice(list(idx_to_human_entity), size=(int(negatives_ratio*len(human_only_tr_pos)), 2))

# mouse_only_dev_neg = np.random.choice(list(idx_to_mouse_entity), size=(int(negatives_ratio*len(mouse_only_dev_pos)), 2))
# human_only_dev_neg = np.random.choice(list(idx_to_human_entity), size=(int(negatives_ratio*len(human_only_dev_pos)), 2))

In [ ]:
# dd = (3062,962)
# print(random.choice(list(idx_to_human_entity)))
# print(idx_to_human_entity[random.choice(list(idx_to_human_entity))])

# x = 3062
# y = 961

# while (x, y) in alignments:
#     y = y + 1
#     print(x,y)

# if dd in alignments:
#     print("hi")

In [ ]:
# np.savetxt('test.tsv', all_edges, delimiter='\t', fmt='%1.1d')

In [ ]:
# idx_to_entity[1748].label

In [ ]:
# aa = [(3,4), (4,3), (5,6), (6,5), (7,8), (8,7)]
# bb = [(3,4), (4,3), (5,6), (6,5), (7,8), (8,7), (8,9), (9,8)]
# cc = [(3,4), (4,3), (5,6), (6,5), (7,8), (8,7), (8,9), (9,8), (3,4), (4,3)]

# print(len(aa)/2%2)
# print(len(bb)/2%2)
# print(len(alignments)/2%2)

# if len(aa)/2%2 == 1.0:
#     print(aa[:int(len(aa)*0.5)+1])
#     print(aa[int(len(aa)*0.5)+1:])
    
# if len(bb)/2%2 == 0:
#     print(bb[:int(len(bb)*0.5)])
#     print(bb[int(len(bb)*0.5):])


## Saving edges within the ontologies

## THE FOLLOWING BLOCK DOES NOT NEED TO BE INCLUDED

In [ ]:
# mouse_edges = []
# human_edges = []


# # loop over all of the entities in the dictionary of mouse entities
# for m in mouse_entities: 
    
#     # loop over the entity's 'is_a' relations
#     for i in m.is_a:         
        
#         # this condition is for UNDEFINED edges
#         if hasattr(i, 'value'): 
#             mouse_edges.append((mouse_entities[m], mouse_entities[i.value]))
#             #print(mouse_entities[h], mouse_entities[i.value])
            
#         # this condition is for subClassOf relations
#         else: 
#             mouse_edges.append((mouse_entities[m], mouse_entities[i]))
#             #print(mouse_entities[h], mouse_entities[i])

            
            
# # loop over all of the entities in the dictionary of human entities
# for h in human_entities: 
    
#     # loop over the entity's 'is_a' relations
#     for i in h.is_a:
        
#         # this condition is for UNDEFINED edges
#         if hasattr(i, 'value'): 
#             human_edges.append((human_entities[h], human_entities[i.value]))
            
#         # this condition is for subClassOf relations
#         else: 
#             human_edges.append((human_entities[h], human_entities[i]))

In [ ]:
# entities_names['Lower_Extremity']
# entities[human.NCI_C12742]
# idx_to_entity[3]
# entities[owlready2.entity.ThingClass('mouse.MA_0001112')]

# idx_to_entity_names[3]

# if 0 in _parents:
#     print('ay')
# else:
#     print('nah')

# _children[0]
# _parents[2738]
# m_children[0]

In [ ]:
# tmp = random.choice(alignments)
# print(tmp)
# print(tmp[0])
# negatives_ratio = 1

# dev_neg_alignments = []

# for i in range(int(len(dev_pos_alignments)/2) * negatives_ratio):
#     rdm_m_idx = random.choice(list(idx_to_mouse_entity))
#     rdm_h_idx = random.choice(list(idx_to_human_entity))
    
#     while (rdm_m_idx, rdm_h_idx) in alignments:
#         rdm_h_idx = random.choice(list(idx_to_human_entity))
        
#     dev_neg_alignments.append((rdm_h_idx, rdm_m_idx))
#     dev_neg_alignments.append((rdm_m_idx, rdm_h_idx))